In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

PATH = '/home/kai/data/kaggle/homecredit/'

train_cleaned = pd.read_csv(PATH+'application_train.csv')

test_cleaned = pd.read_csv(PATH+'application_test.csv')

# train_cleaned = pd.read_pickle(PATH + 'inter/train_only_0.pkl')
# test_cleaned = pd.read_pickle(PATH + 'inter/test_only_0.pkl')
print('done')

done


In [2]:
target = train_cleaned['TARGET']
ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR', 'tmp_idx']
categorical_feats = [x for x in train_cleaned.columns if train_cleaned[x].dtype == 'object']
categorical_feats = [x for x in categorical_feats if x not in ignore_cols]
features = [x for x in train_cleaned.columns if x not in ignore_cols + categorical_feats]

train = train_cleaned[features]
test = test_cleaned[features]

In [3]:
print(train.shape, test.shape)

(307511, 104) (48744, 104)


In [4]:
%%time

from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

lgbm_train = lgbm.Dataset(data=train,
                          label=target,
                          categorical_feature=[],#categorical_feats,
                          free_raw_data=False)

# lgbm_params = {'num_threads': 16, 'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
#           'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 10000, 'verbose': 100,
#           'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
#           'min_split_gain':.01, 'min_child_weight':1}

# lgbm_params ={ 'nthread' : 8,
#         'n_estimators':10000,
#         'learning_rate':0.02,
#               'num_rounds':500,
#         'num_leaves':34,
#         'colsample_bytree':0.9497036,
#         'subsample':0.8715623,
#         'max_depth':8,
#         'reg_alpha':0.041545473,
#         'reg_lambda':0.0735294,
#         'min_split_gain':0.0222415,
#         'min_child_weight':39.3259775,
#         'silent':-1,
#         'verbose':-1, }
lgbm_params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 500, 'verbose': 0 ,
          'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
          'min_split_gain':.01, 'min_child_weight':1,'num_threads': 12,'feature_fraction': 0.9}

cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=5,
                     early_stopping_rounds=150,
                     verbose_eval=100,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[100]	cv_agg's auc: 0.730152 + 0.00350847
[200]	cv_agg's auc: 0.733479 + 0.00364645
[300]	cv_agg's auc: 0.737691 + 0.00362347
[400]	cv_agg's auc: 0.742216 + 0.00333477
[500]	cv_agg's auc: 0.746401 + 0.00330098
Optimum boost rounds = 499
Best CV result = 0.7464006329994687
CPU times: user 20min 23s, sys: 6.07 s, total: 20min 29s
Wall time: 1min 45s


In [5]:
# clf = lgbm.train(train_set=lgbm_train,
#                  params=lgbm_params,
#                  num_boost_round=int(optimum_boost_rounds*1.1))
clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=int(optimum_boost_rounds*1))

""" Predict on test set and create submission """
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_cleaned['SK_ID_CURR'], 'TARGET': y_pred})
print('done')

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


done


In [9]:
(y_pred >0.5).sum()

21

# submission

In [ ]:
print((out_df['TARGET']>0.5).sum())

out_df.to_csv(PATH + 'submission/6_21_train.csv', index=False)
print('done')

In [ ]:
out_df[out_df['TARGET']< 0]

# Display Importance

In [ ]:
import matplotlib.pyplot as plt
out_df[['TARGET']].hist()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, (ax, ax1) = plt.subplots(1, 2, figsize=[15, 7])
lgbm.plot_importance(clf, ax=ax, max_num_features=30, importance_type='split')
lgbm.plot_importance(clf, ax=ax1, max_num_features=30, importance_type='gain')
ax.set_title('Importance by splits')
ax1.set_title('Importance by gain')
plt.tight_layout()
plt.show()

# Display Importance

In [ ]:
import json

y = [x for x in zip(clf.feature_name(), clf.feature_importance('gain'))]
y.sort(key=lambda x: x[1], reverse=True)
y = [(i, j) for i, j in enumerate(y)]
y
# with open('./importance_gain.json') as f:
#     json.dump(x, f)

In [ ]:
x = [x for x in zip(clf.feature_name(), clf.feature_importance('split'))]
x.sort(key=lambda x: x[1], reverse=True)
x = [(i, j) for i, j in enumerate(x)]
x
# with open('./importance_split.json') as f:
#     json.dump(x, f)